# Homework 3 

You will have 2 datasets to work on. 
#### You have to answer questions in this [form](https://goo.gl/forms/5gfxvKZxpoydoeOB2) and provide your code

### 1. Wikipedia Web Traffic Time Series

Data from [Kaggle competition](https://www.kaggle.com/c/web-traffic-time-series-forecasting)* )

*wikipedia_train3* - train data *wikipedia_test3* - test data created by us from original train data . For more information about dataset, please visit Homework1 assignment


### 2. French dataset

Target variable is prime_tot_ttc – house of insurance price.<p>
The variables in the files are:<p>
code_postal: postal code of the insured property<p>
nb_pieces: number of rooms<p>
veranda: presence of a veranda<p>
biens_nomades: ceiling of coverage of goods (laptop, tablets, musical instruments ,…)<p>
sit_fam: family situation of the insured<p>
profession: profession of the insured<p>
nature_lri: nature of the insured property<p>
societe: company covering the risk<p>
cdomact: area of activity of the insured<p>
niv_urbanisation: level of urbanization in which is the insured property<p>
age_societaire: age of the member<p>
objets_precieux: presence of precious objects in the insured property<p>
pres_contrat_auto: holding a contract of auto insurance<p>
formule: guarantee formula<p>
sit_juridique: legal situation<p>
surf_habitat: area of insured property<p>
patrimoine_mob: amount of heritage insured furniture<p>
var1 - ... - var11: non-explicit variables<p>


## Wikipedia page views (SMAPE metric)

In [1]:
import pandas as pd
import numpy as np
import math
from dateutil.relativedelta import relativedelta

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [2]:
### This is data from wikipedia3 archive in data folder

train = pd.read_csv("data/wikipedia_train3.csv")
test = pd.read_csv("data/wikipedia_test3.csv")

In [15]:
train['date'] = pd.to_datetime(train['date'])

**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. **For _only_ train** create a holdout validation using any type of split you think is useful here. What is the split type you are using? Answer in google forms

**2.** Write a code to compare the score of your validation and test set. For scoring, use metric SMAPE (code is in lecture). For prediction use 15 previous days median. In the google form write your validation score.

In [12]:
print(train.values[0][1])
print(train.values[-1][1])

print(test.values[0][1])
print(test.values[-1][1])

2016-01-01
2016-08-31
2016-09-10
2016-11-10


In [ ]:
# print(pd.to_datetime('2016-06-25').weekday())
# print(pd.to_datetime('2016-09-10').weekday())
# print(pd.to_datetime('2016-11-10').weekday())
# print(pd.to_datetime('2016-08-25').weekday())

In [54]:
date1 = pd.to_datetime('2016-06-15')
date2 = pd.to_datetime('2016-06-25')
date2_e = pd.to_datetime('2016-08-25')

In [55]:
ntrain = train[train['date']<date1]
valid = train[(train['date']>=date2) & (train['date']<=date2_e)]

In [56]:
def smape_fast(y_true, y_pred):
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true.iloc[i]
        b = y_pred[i]
        if b < 1:
            b = 0
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out

In [120]:
median_15 = ntrain.loc[ntrain['date'] >= '2016-06-01'].groupby(['Page']).median().reset_index()
median_15.rename(columns={"Visits": "pred_Visits"}, inplace=True)

In [59]:
y = valid['Visits']
y_pred = valid.merge(median_15, on="Page", how='left')['pred_Visits']

smape_fast(y, y_pred)

45.556450965019096

In [61]:
# # i'm probably should comment this cell)
# median_15 = train.loc[train['date'] >= '2016-08-15'].groupby(['Page']).median().reset_index()
# median_15.rename(columns={"Visits": "pred_Visits"}, inplace=True)

In [121]:
y = test['Visits']
y_pred = test.merge(median_15, on="Page", how='left')['pred_Visits']

smape_fast(y, y_pred)

49.35397697233633

**3.** Perform K-fold validation using your type of split. Run GridSearch with any classificator you like and set of parameters to optimize, providing it with your custom validation.  Compare the score of your validation and test set. For scoring, again, use metrics SMAPE. In the google form write your scores on validation and test sets

In [65]:
tmonths = 2
vmonths = 2

def create_validation(df, start_date):
    return df.loc[(df['date'] >= pd.to_datetime(start_date) - relativedelta(days=0)) & \
                  (df['date'] <  pd.to_datetime(start_date) + relativedelta(months=tmonths))].index, \
           df.loc[(df['date'] >= pd.to_datetime(start_date) + relativedelta(days=10) + relativedelta(months=tmonths)) & \
                  (df['date'] <  pd.to_datetime(start_date) + relativedelta(days=10) + relativedelta(months=tmonths+vmonths))].index

In [71]:
dates = ['2016-02-15', '2016-03-15', '2016-04-15']

In [72]:
### Your validation iterator
CVIterator = []

for i in dates:
    trainIndices, valIndices = create_validation(train, i)
    CVIterator.append( (trainIndices, valIndices) )

In [110]:
from sklearn.base import BaseEstimator, RegressorMixin

class MedianReg(BaseEstimator, RegressorMixin):  
    """Median Regression"""

    def __init__(self, days=15):
        """
        Called when initializing the regression
        """
        self.days = days


    def fit(self, X, y=None):
        first_date = pd.to_datetime(X['date'].values[0]) - relativedelta(months=self.days)
        
        medians = X.loc[X['date'] >= first_date].groupby(['Page']).median().reset_index()
        medians.rename(columns={"Visits": "pred_Visits"}, inplace=True)

        self.medians = medians
        
        return self

    def predict(self, X, y=None):
        y_pred = X.merge(self.medians, on="Page", how='left')['pred_Visits']
        self.pred_ = y_pred
        return y_pred

    def score(self, X, y=None):
        # counts number of values bigger than mean
        y_pred = X.merge(self.medians, on="Page", how='left')['pred_Visits']
        score = smape_fast(y, y_pred)
#         print(score)
        return score


In [113]:
param_grid = {
#     "weekends": [False, True],    
    "days": [7, 10, 12, 14, 25, 30]
}

grid_search = GridSearchCV(MedianReg(), cv=CVIterator, param_grid=param_grid)

In [116]:
grid_search.fit(train, train['Visits'])

GridSearchCV(cv=[(Int64Index([ 531675,  531676,  531677,  531678,  531679,  531680,  531681,
             531682,  531683,  531684,
            ...
            1240565, 1240566, 1240567, 1240568, 1240569, 1240570, 1240571,
            1240572, 1240573, 1240574],
           dtype='int64', length=708900), Int64Ind...2800150, 2800151,
            2800152, 2800153, 2800154],
           dtype='int64', length=720715))],
       error_score='raise', estimator=MedianReg(days=15), fit_params=None,
       iid=True, n_jobs=1, param_grid={'days': [7, 10, 12, 14, 25, 30]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [117]:
grid_search.best_estimator_

MedianReg(days=7)

In [118]:
grid_search.best_score_

47.13523963290248

In [119]:
grid_search.best_estimator_.score(test, test['Visits'])

42.70966463467286

## French house insurance prediction (MAPE metric)

**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. You may use any tool you like, any EDA you want. Decribe your validation and reason for your choice in google form.

**2.** 
+ Implement your validation. 
+ Generate additional features. 
+ Build a model you like/you can. 
+ Wait for training your model. If it takes too much time, listen to https://www.youtube.com/watch?v=XWkGL4eZR3E
+ Submit your scores on validation and test to google form. Use MAPE metric for scoring. 
+ Ланфрен ланфра лантатита

In [128]:
import pandas_profiling as pp

In [144]:
train = pd.read_csv('data/train_french.csv')
test = pd.read_csv('data/test_french.csv')

In [146]:
y_train = train['prime_tot_ttc']
y_test = test['prime_tot_ttc']
train = train.drop(['prime_tot_ttc'], axis=1)
test = test.drop(['prime_tot_ttc'], axis=1)

In [137]:
train.columns

Index(['code_postal', 'nb_pieces', 'veranda', 'biens_nomades', 'sit_fam',
       'profession', 'nature_lri', 'societe', 'domaine_activite',
       'niv_urbanisation', 'age_societaire', 'objets_precieux',
       'pres_contrat_auto', 'formule', 'sit_juridique', 'surf_habitat',
       'patrimoine_mob', 'var1', 'var2', 'var3', 'var4', 'var5', 'var6',
       'var7', 'var8', 'var9', 'var10', 'var11', 'prime_tot_ttc'],
      dtype='object')

In [142]:
train.head()

,code_postal,nb_pieces,biens_nomades,societe,domaine_activite,age_societaire,objets_precieux,pres_contrat_auto,formule,sit_juridique,...,niv_urbanisation_A,niv_urbanisation_B,niv_urbanisation_C,niv_urbanisation_D,niv_urbanisation_E,niv_urbanisation_F,niv_urbanisation_G,niv_urbanisation_~,var9_NON,var9_OUI
0,62510,1,500,2,34,35,1,0,1,0,...,1,0,0,0,0,0,0,0,1,0
1,83126,2,500,2,37,27,1,1,2,0,...,1,0,0,0,0,0,0,0,1,0
2,68224,3,500,1,1,24,1,1,2,0,...,0,0,0,0,1,0,0,0,1,0
3,30263,3,500,2,34,39,1,1,1,1,...,1,0,0,0,0,0,0,0,1,0
4,72231,2,500,1,19,80,1,1,2,1,...,0,0,0,1,0,0,0,0,1,0


In [147]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [140]:
from sklearn.model_selection import train_test_split

In [148]:
X_train, X_test, y_train, y_test = train_test_split(
     train, y_train, test_size=0.33, random_state=42)

In [150]:
import lightgbm as lgb

In [151]:
params = {"objective": "mape",
        "metric":"mape",
        "reg_sqrt":True,
        "num_leaves": 50,
        }

num_rounds = 1000

In [ ]:
lgb.

In [134]:
# pp.ProfileReport(train)

In [133]:
# pp.ProfileReport(test)